# BHT Data Applications project
# Automatic Anime recommendation Algorithm
### This project aims to create an algorithm that can determine what anime to recommend to a user.
##### Authors: Rashmi Di Michino and Antonin Mathubert

The 320000 users and 16000 animes dataset was taken from https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020 <br>
We are going to use this dataset to build a model that can recommend an anime based on the animes that the user is watching, has dropped, has kept on hold or put on their watching list.

### 1. Importing and parsing the data
First, we want to import all of our available data in a suitable manner so it is treatable for the next steps of the project.<br><br>
In order to load the data, we are going to do it by chunking the csv file so it's more efficient. Then we're changing the default type of the columns to be more convenient memory wise.

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
#import cupy as cp
import random
import time
import os
import re

In [ ]:
dataset_chunks = pd.read_csv("dataset/anime/animelist.csv", chunksize=10000)
chunks = []
for chunk in dataset_chunks:
    chunks.append(chunk)
    
dataset = pd.concat(chunks, ignore_index=True)
dataset = dataset.astype({'user_id': "int32", 'anime_id': 'int32', "watching_status": "int16"})

dataset_chunks = None
chunks = None

In [ ]:
dataset_chunks = pd.read_csv("C:/Users/rashm/OneDrive/Desktop/data_applications_project/julius/anime_dataset/animelist.csv", chunksize=10000)

chunks = []
for chunk in dataset_chunks:
    chunks.append(chunk)
    
dataset = pd.concat(chunks, ignore_index=True)
dataset = dataset.astype({'user_id': "int32", 'anime_id': 'int32', "watching_status": "int16"})

dataset_chunks = None
chunks = None

### 2. Recommendation system based on the watched animes
In this first version we're going to implement a recommendation system based on which animes the users have seen, for example if someone has watched cowboy bepop, they're going to be recommended to see death note
#### Reducing the dataset
As the dataset we're working with is too large, we're going to reduce it

In [ ]:
dataset.drop(['rating', 'watched_episodes'], axis=1, inplace=True)
dataset = dataset[(dataset['anime_id'] < 10000) & (dataset['user_id'] < 20000)]
dataset = dataset[(dataset['user_id'] != 61960) & (dataset['watching_status'] != 4)]
dataset = dataset.drop("watching_status", axis=1)

Here we can see a sample of how the dataset is structured

In [ ]:
display(dataset.head(100))
len(dataset)

The next step is pivoting the dataset: we're constructing a matrix that will be used to build the recommendation system, where the rows are the users' ids and the columns are the animes' ids.

In [ ]:
dataset = dataset.pivot(index='user_id', columns='anime_id', values='anime_id')

We are now converting our matrix into a binary matrix in order to be able to retrieve the association rules

In [ ]:
dataset[dataset.notnull()] = True
dataset = dataset.fillna(False)

Finally, we are exploiting the mlxtend library to build the recommendation system and we're retrieving the association rules

In [ ]:
frequent_itemsets  = apriori(dataset, use_colnames=True, min_support=0.2)

frequent_itemsets

rules = association_rules(frequent_itemsets)

rules

By running this next cell we can see that for the users that have seen Cowboy Bepop it's recommended to see Death Note

In [ ]:
rules["antecedents"] = rules["antecedents"].apply(lambda x: [x for x in x])
value = 1535
rules[rules["antecedents"].apply(lambda x: value in x)]

In [ ]:
dataset = np.array(dataset.values)

In [ ]:
dataset = np.nan_to_num(dataset, nan=0)

In [ ]:
dataset = np.where(dataset != 0, 1, dataset)